## Metricas textuarales de Haralick 

In [3]:
import rasterio
import numpy as np
import dask.array as da
from dask.diagnostics import ProgressBar
from mahotas.features import haralick

# 1) Parámetros
INPUT = r"NDVI_mean.tif"
OUTS = {
    "contrast":    r"haralick_contrast.tif",
    "homogeneity": r"haralick_homogeneity.tif",
    "energy":      r"haralick_energy.tif",
    "correlation": r"haralick_correlation.tif",
}
WINDOW = 7              # ventana 7×7
PAD    = WINDOW // 2    # solapamiento
LEVELS = 32             # número de niveles para la GLCM
CHUNKS = (512, 512)     # tamaño de bloque Dask

# 2) Leer, cuantizar y convertir en dask.array
with rasterio.open(INPUT) as src:
    nodata = src.nodata
    arr = src.read(1).astype(float)
    meta = src.meta.copy()
# cuantizar entre 0 y LEVELS-1
mn, mx = np.nanmin(arr), np.nanmax(arr)
q = ((arr - mn) / (mx - mn) * (LEVELS-1)).clip(0, LEVELS-1).astype(np.uint8)
# enmascarar nodata
q[arr == nodata] = 0
darr = da.from_array(q, chunks=CHUNKS)

# 3) Función de cálculo Haralick para un solo bloque
def haralick_block(block, feature_index):
    out = np.zeros_like(block, dtype=np.float32)
    for i in range(PAD, block.shape[0] - PAD):
        for j in range(PAD, block.shape[1] - PAD):
            win = block[i-PAD:i+PAD+1, j-PAD:j+PAD+1]
            feats = haralick(win, distance=1, return_mean=True)
            out[i, j] = feats[feature_index]
    # recortar los bordes de solapamiento
    return out[PAD:-PAD or None, PAD:-PAD or None]

# 4) Mapear cada métrica con map_overlap
feature_indices = {
    "contrast":    0,
    "correlation": 1,
    "energy":      2,
    "homogeneity": 3
}

for name, feat_idx in feature_indices.items():
    # aplica la función en paralelo con solapamiento PAD
    mapped = darr.map_overlap(
        lambda blk: haralick_block(blk, feat_idx),
        depth=PAD, boundary="reflect", dtype=np.float32
    )
    # computa el array completo
    with ProgressBar():
        result = mapped.compute()

    # 5) Guarda resultado
    meta.update(dtype=rasterio.float32, count=1, nodata=None)
    with rasterio.open(OUTS[name], "w", **meta) as dst:
        dst.write(result, 1)

    print(f"– {name} listo en {OUTS[name]}")


[########################################] | 100% Completed | 73.96 ss
– contrast listo en haralick_contrast.tif
[########################################] | 100% Completed | 70.63 ss
– correlation listo en haralick_correlation.tif
[########################################] | 100% Completed | 67.68 ss
– energy listo en haralick_energy.tif
[########################################] | 100% Completed | 71.42 ss
– homogeneity listo en haralick_homogeneity.tif


### Otras métricas

In [17]:
import rasterio
import numpy as np
from rasterio.windows import Window
from skimage.feature import hog
from skimage.filters import gabor

# Parámetros
INPUT       = r"NDVI_mean.tif"
OUT_HOG     = r"hog_map.tif"
OUT_LESHi   = r"lesh_{freq:.2f}_{theta:.2f}.tif"

WINDOW      = 4      # tamaño de la ventana móvil (debe caber completo)
STEP        = 1      # salto (stride)
HOG_PARAMS  = dict(
    orientations=8,
    pixels_per_cell=(16, 16),
    cells_per_block=(1, 1),
    feature_vector=True
)
GABOR_FREQS = [0.1, 0.2]
GABOR_THETAS= [0, np.pi/4, np.pi/2, 3*np.pi/4]

# Abrir origen y destinos
with rasterio.open(INPUT) as src:
    profile = src.profile.copy()
    nodata = src.nodata
    height, width = src.height, src.width

    # Preparamos perfil de salida (una banda float32)
    profile.update(dtype=rasterio.float32, count=1, nodata=None)

    # --- HOG RASTER ---
    with rasterio.open(OUT_HOG, 'w', **profile) as dst_hog:
        # Recorremos solo posiciones donde cabe WINDOW×WINDOW
        for i in range(0, height - WINDOW + 1, STEP):
            for j in range(0, width  - WINDOW + 1, STEP):
                win = Window(j, i, WINDOW, WINDOW)
                block = src.read(1, window=win).astype(float)

                # Saltar si todo es nodata
                if np.all(block == nodata):
                    continue

                # Normalizar y manejar NaN
                block[block == nodata] = np.nan
                mn, mx = np.nanmin(block), np.nanmax(block)
                # Evitar división por cero
                if mx == mn:
                    norm = np.zeros_like(block, dtype=float)
                else:
                    norm = (block - mn) / (mx - mn)

                # Calcular HOG y tomar la media
                feats = hog(norm, **HOG_PARAMS)
                val   = np.nanmean(feats).astype(np.float32)

                # Escribir ese valor constante en la ventana
                out_block = np.full((WINDOW, WINDOW), val, dtype=np.float32)
                dst_hog.write(out_block, 1, window=win)

    # --- LESH RASTERS (Gabor energy) ---
    for freq in GABOR_FREQS:
        for theta in GABOR_THETAS:
            outpath = OUT_LESHi.format(freq=freq, theta=theta)
            with rasterio.open(outpath, 'w', **profile) as dst_lesh:
                for i in range(0, height - WINDOW + 1, STEP):
                    for j in range(0, width  - WINDOW + 1, STEP):
                        win = Window(j, i, WINDOW, WINDOW)
                        block = src.read(1, window=win).astype(float)

                        if np.all(block == nodata):
                            continue

                        block[block == nodata] = np.nan
                        mn, mx = np.nanmin(block), np.nanmax(block)
                        if mx == mn:
                            norm = np.zeros_like(block, dtype=float)
                        else:
                            norm = (block - mn) / (mx - mn)

                        # Filtrado Gabor y energía media
                        real, imag = gabor(norm, frequency=freq, theta=theta)
                        mag = np.sqrt(real**2 + imag**2)
                        val = np.nanmean(mag).astype(np.float32)

                        out_block = np.full((WINDOW, WINDOW), val, dtype=np.float32)
                        dst_lesh.write(out_block, 1, window=win)


C:\Users\jorgh\AppData\Local\Temp\ipykernel_28036\2100211387.py:55: RuntimeWarning: Mean of empty slice
  val   = np.nanmean(feats).astype(np.float32)


In [18]:
import rasterio
import numpy as np
from rasterio.windows import Window
from skimage.measure import moments, moments_hu

# Parámetros
INPUT    = r"NDVI_mean.tif"
OUTPUT   = r"hu_moments.tif"
WINDOW   = 4
STEP     = 1
N_MOM    = 7

# Abre el ráster de entrada
with rasterio.open(INPUT) as src:
    profile = src.profile.copy()
    nodata  = src.nodata
    h, w    = src.height, src.width

    # Prepara un GeoTIFF de 7 bandas (uno por cada Hu moment)
    profile.update(
        count=N_MOM,
        dtype=rasterio.float32,
        nodata=None
    )

    with rasterio.open(OUTPUT, 'w', **profile) as dst:
        for i in range(0, h - WINDOW + 1, STEP):
            for j in range(0, w - WINDOW + 1, STEP):
                win   = Window(j, i, WINDOW, WINDOW)
                block = src.read(1, window=win).astype(float)

                # Saltar ventanas con sólo nodata
                if np.all(block == nodata):
                    continue

                # Normalizar a [0,255] para moments
                block[block == nodata] = np.nan
                mn, mx = np.nanmin(block), np.nanmax(block)
                if mx == mn:
                    img = np.zeros_like(block, dtype=np.uint8)
                else:
                    img = ((block - mn) / (mx - mn) * 255).astype(np.uint8)

                # Calcula momentos y Hu moments
                M   = moments(img)               # momentos de orden arbitrario
                hu  = moments_hu(M)              # array de 7 Hu moments

                # Log-transformación estándar
                hu_log = -np.sign(hu) * np.log10(np.abs(hu) + 1e-12)

                # Escribe cada banda
                for b in range(N_MOM):
                    band = np.full((WINDOW, WINDOW), hu_log[b], dtype=np.float32)
                    dst.write(band, b+1, window=win)

print("¡Ráster de momentos de Hu (7 bandas) creado correctamente!")


¡Ráster de momentos de Hu (7 bandas) creado correctamente!
